In [1]:
import io, os, sys, types

In [2]:
import numpy as np

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days


In [4]:
np.array([1,1])

array([1, 1])

$$\alpha = 5$$

In [3]:
import multiprocessing as mp

In [4]:
mp.cpu_count()

8

In [1]:
import numpy as np

In [6]:
from multiprocessing import Pool
import time
import numpy
 
def takeuptime(ntrials):
    for ii in xrange(ntrials):
        junk = numpy.std(numpy.random.randn(1e5))
    return junk
 
if __name__ == "__main__":
    start = time.time()
    map(takeuptime, [100000,100000])
    print "Serial time: %f" % (time.time() - start)
 
    start = time.time()
    pool = Pool(processes=8)
    pool.map(takeuptime, [100000, 100000])
    print "Parallel time: %f" % (time.time() - start)

Serial time: 135.262062


/home/john/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Parallel time: 75.328563


In [3]:
import ipyparallel

In [4]:
clients = ipyparallel.Client()
clients.ids

[0, 1, 2, 3, 4, 5, 6, 7]

In [5]:
dview = clients.direct_view()
with dview.sync_imports():
    import numpy  #using 'as np' does not work!

importing numpy on engine(s)


In [15]:
def mc_pi(n):
    count = 0
    for i in range(n):
        x = numpy.random.rand()
        y = numpy.random.rand()
        if (x**2 + y**2) <= 1:
            count += 1
    return 1.0 * count/n

In [14]:
start = time.time()
est_pi = 4.0 * mc_pi(10**8)
print "Serial time: %f" % (time.time() - start)
print "Pi = %f" % est_pi

Serial time: 53.868618
Pi = 3.141501


In [142]:
results = dview.map_sync(mc_pi, [125*10**5]*8)
print(results)

[0.78564736, 0.78527352, 0.78545552, 0.78540392, 0.7853624, 0.78537504, 0.7855432, 0.78551376]


In [16]:
n_proc = 8;
n = 10**8
n_pass = n/n_proc
est_pis = dview.map_sync(mc_pi, [n_pass]*n_proc)
print(4.0 * sum(est_pis)/n_proc)

3.14191492


We should be able to import variables across engines if we import the function with dview.sync_imports()

In [7]:
def funct(nstart,nend):
    
    def iter_loadtxt(filename, delimiter=',', skiprows=1, dtype=float):
        def iter_func():
            with open(filename, 'r') as infile:
                for _ in range(skiprows):
                    next(infile)
                for line in infile:
                    line = line.rstrip().split(delimiter)
                    for item in line:
                        yield dtype(item)
            iter_loadtxt.rowlength = len(line)

        data = numpy.fromiter(iter_func(), dtype=dtype)
        data = data.reshape((-1, iter_loadtxt.rowlength))
        return data
    
    file1 = iter_loadtxt('/media/john/My Passport/slabs/serge_tests/data0.csv',delimiter=',',skiprows=1,dtype=float)
    polar0 = [row[0] for row in file1]
    polar1 = [row[1] for row in file1]
    polar2 = [row[2] for row in file1]
    nodeID = [row[3] for row in file1]
    r0 = [row[5] for row in file1]
    r1 = [row[6] for row in file1]
    r2 = [row[7] for row in file1]
    
    polar0ave = numpy.mean(polar0);
    polar1ave = numpy.mean(polar1);
    polar2ave = numpy.mean(polar2);
  
    Cr = numpy.zeros(len(polar0));
    R = numpy.zeros(len(polar0));
    for i in range(1,len(polar0)):
        for j in range(nstart,nend):
            if i != j:
                Cr[i] += 1/(0.8*0.8)*((polar0[i] - polar0ave)* (polar0[j] - polar0ave) + (polar1[i] - polar1ave) * (polar1[j] - polar1ave) + (polar2[i] - polar2ave) * (polar2[j] - polar2ave));
        R[i] = (r0[i]*r0[i] +  r1[i]*r1[i] + r2[i]*r2[i])**(0.5);
    return [Cr, R];
    

In [83]:
funct(10).shape

AttributeError: 'tuple' object has no attribute 'shape'

In [116]:
import time

start = time.time()
n_proc = len(clients.ids);
n = 4000; #should be multiple of n_proc
dof_pass = n/n_proc;

Crfunc = dview.map_sync(funct,range(0,n,dof_pass),range(0 + dof_pass,n + 1,dof_pass))

print "Parallel time: %f" % (time.time() - start)
#the above code uses two input arguments (two ranges) for the distribution of the outer loop

#how do call/save the solution? first need to catenate them -- Cr = Crfunc[pid][0], R = Crfunc[pid][1];

Parallel time: 235.973917


In [111]:
#4 processor version:

CR = numpy.concatenate(
   (
    Crfunc[0][0][numpy.nonzero(Crfunc[0][0])], 
    Crfunc[1][0][numpy.nonzero(Crfunc[1][0])],
    Crfunc[2][0][numpy.nonzero(Crfunc[2][0])],
    Crfunc[3][0][numpy.nonzero(Crfunc[3][0])]
)
                               );

R = numpy.concatenate(
   (
    Crfunc[0][1][numpy.nonzero(Crfunc[0][1])], 
    Crfunc[1][1][numpy.nonzero(Crfunc[1][1])],
    Crfunc[2][1][numpy.nonzero(Crfunc[2][1])],
    Crfunc[3][1][numpy.nonzero(Crfunc[3][1])]
)
                               );